<a href="https://colab.research.google.com/github/pratikiiitb2013/END2_p1/blob/main/Session3/%20END2_S3_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from random import randint

In [2]:
# def get_device():
#     if torch.cuda.is_available():
#         device = 'cuda:0'
#     else:
#         device = 'cpu'
#     return device
# device = get_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# sample_data = np.arange(0, 10)
# print('The whole data: ', sample_data)
# dataset = ExampleDataset(sample_data)
# print('Number of samples in the data: ', len(dataset))
# print(dataset[2])
# print(dataset[0:5])

In [4]:
!wget https://pjreddie.com/media/files/mnist_train.csv
!wget https://pjreddie.com/media/files/mnist_test.csv

--2021-05-21 13:39:31--  https://pjreddie.com/media/files/mnist_train.csv
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109575994 (104M) [application/octet-stream]
Saving to: ‘mnist_train.csv’

mnist_train.csv     100%[===================>] 104.50M  35.3MB/s    in 3.0s    

2021-05-21 13:39:34 (35.3 MB/s) - ‘mnist_train.csv’ saved [109575994/109575994]

--2021-05-21 13:39:34--  https://pjreddie.com/media/files/mnist_test.csv
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18289443 (17M) [application/octet-stream]
Saving to: ‘mnist_test.csv’

mnist_test.csv      100%[===================>]  17.44M  24.3MB/s    in 0.7s    

2021-05-21 13:39:35 (24.3 MB/s) - ‘mnist_test.csv’ saved [18289443/18289443]



In [5]:
!ls

mnist_test.csv	mnist_train.csv  sample_data


In [6]:
df_train = pd.read_csv('mnist_train.csv', header=None)
df_test = pd.read_csv('mnist_test.csv', header=None)

# get the image pixel values and labels
# train_labels = df_train.iloc[:, 0]
# train_images = df_train.iloc[:, 1:]
# test_labels = df_test.iloc[:, 0]
# test_images = df_test.iloc[:, 1:]

In [7]:
# custom dataset

transform=transforms.Compose([
    transforms.ToTensor()
])

class customMNISTDataset(Dataset):
  def __init__(self, data, transforms=None):
    self.X = data.iloc[:, 1:]
    self.y = data.iloc[:, 0]
    self.transforms = transforms

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    img_data = self.X.iloc[idx, :]
    # if isinstance(idx, slice):
    #     img_data = np.array([i.reshape(1,28, 28) for i in np.array(img_data)])
    #     rand_no = np.array([randint(0, 9) for i in np.array(img_data)])
    #     y1 = np.array(self.y[idx])
    #     y2 = y1 + rand_no
    # else:
    img_data = np.array(img_data).astype(np.uint8).reshape(28, 28, 1)
    rand_no = randint(0, 9)
    y1 = self.y[idx]
    y2 = y1 + rand_no
    
    if self.transforms:
      img_data = self.transforms(img_data)
    return img_data, rand_no, y1, y2




train_ds = customMNISTDataset(df_train, transform)
test_ds = customMNISTDataset(df_test, transform)
print('Number of samples in the train data: ', len(train_ds))
print('Number of samples in the test data: ', len(test_ds))

trainloader = DataLoader(train_ds, batch_size=10, shuffle=True)
aa = next(iter(trainloader))
# aa[1].type()
# print(dataset[2])
# print(dataset[0:5])

# batch = next(iter(trainloader))

# len(batch), type(batch)

Number of samples in the train data:  60000
Number of samples in the test data:  10000


In [8]:
import torch.nn.functional as F

def conv_block(input_size, output_size, kernel_size):
    block = nn.Sequential(
        nn.Conv2d(in_channels=input_size, out_channels=output_size, kernel_size=kernel_size), nn.ReLU(), nn.MaxPool2d((2, 2)),
    )
    return block

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = conv_block(input_size=1, output_size=6, kernel_size=3)
    self.conv2 = conv_block(input_size=6, output_size=12, kernel_size=3)

    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(in_features=12*5*5, out_features=16)
    self.fc2 = nn.Linear(in_features=16, out_features=5)

    self.fc3 = nn.Linear(in_features=10, out_features=10)
    self.fc4 = nn.Linear(in_features=10, out_features=5)

    self.fc5 = nn.Linear(in_features=10, out_features=50)
    self.fc6 = nn.Linear(in_features=50, out_features=29)


  def forward(self, img, ohe):

    img = self.conv1(img)
    img = self.conv2(img)
    img = img.reshape(img.shape[0], -1)  # img = img.reshape(-1, 12*5*5)
    img = self.relu(self.fc1(img))
    img = self.relu(self.fc2(img))

    ohe = self.relu(self.fc3(ohe))
    ohe = self.relu(self.fc4(ohe))
    
    x = torch.cat((img, ohe), dim=1)
    x = self.relu(x)
    x = self.relu(self.fc5(x))
    x = self.fc6(x)
    return x[:,0:10],x[:,10:]
    # return F.softmax(x[:,0:10], dim=1),F.softmax(x[:,10:], dim=1)
    




In [9]:
sample = next(iter(train_ds)) 
image, randno, label, sm = sample

print(image.shape, image.unsqueeze(0).shape, randno, label, sm)

torch.Size([1, 28, 28]) torch.Size([1, 1, 28, 28]) 7 5 12


In [10]:
network = Network()
pred1,pred2 = network(image.unsqueeze(0), F.one_hot(torch.tensor(randno).unsqueeze(0), num_classes=10).float())
print(pred1.sum(), pred2.sum())
pred1,pred2


tensor(0.3265, grad_fn=<SumBackward0>) tensor(-0.2265, grad_fn=<SumBackward0>)


(tensor([[-0.0612,  0.2506,  0.0965, -0.1181,  0.1350,  0.0673, -0.0616,  0.0418,
           0.0790, -0.1029]], grad_fn=<SliceBackward>),
 tensor([[ 0.0967, -0.1094, -0.1569, -0.1711,  0.0335, -0.1528, -0.0331,  0.0831,
           0.0807,  0.0204, -0.0438,  0.2704,  0.0534, -0.0263, -0.1500,  0.0234,
          -0.0539,  0.0748, -0.0660]], grad_fn=<SliceBackward>))

In [11]:
train_loader = torch.utils.data.DataLoader(
    train_ds, 
    batch_size=32
)

batch = next(iter(train_loader))
print(len(batch))
batch

images, randnos, labels, sums = batch

pred1,pred2 = network(images, F.one_hot(torch.tensor(randnos), num_classes=10).float())
print(pred1.sum(dim = 1), pred2.sum(dim = 1))
print(pred1.shape,pred2.shape)
pred1.argmax(dim=1).eq(labels)

4
tensor([0.3111, 0.3383, 0.2910, 0.3487, 0.3480, 0.3086, 0.3154, 0.3501, 0.3280,
        0.3471, 0.3273, 0.3259, 0.3263, 0.3079, 0.3550, 0.3064, 0.3523, 0.3456,
        0.3521, 0.2966, 0.3534, 0.3219, 0.3071, 0.2933, 0.3417, 0.3467, 0.3092,
        0.3384, 0.2885, 0.3272, 0.2988, 0.2885], grad_fn=<SumBackward1>) tensor([-0.2285, -0.2356, -0.2447, -0.2268, -0.2228, -0.2296, -0.2344, -0.2212,
        -0.2335, -0.2229, -0.2293, -0.2276, -0.2310, -0.2334, -0.2266, -0.2310,
        -0.2165, -0.2201, -0.2291, -0.2374, -0.2130, -0.2364, -0.2304, -0.2384,
        -0.2257, -0.2264, -0.2327, -0.2282, -0.2362, -0.2291, -0.2419, -0.2433],
       grad_fn=<SumBackward1>)
torch.Size([32, 10]) torch.Size([32, 19])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


tensor([False, False, False,  True, False, False,  True, False,  True, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False,  True,  True, False, False, False, False, False,
        False, False])

In [12]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [13]:
model = Network().to(device)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=4)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(100):

    total_loss = 0
    total_loss_t = 0
    total_correct_images = 0
    total_correct_sums = 0
    total_correct_images_t = 0
    total_correct_sums_t = 0
    
    # train the model #
    model.train()
    for batch in train_loader: # Get Batch
        images, randnos, labels, sums = batch
        images, randnos, labels, sums = images.to(device), randnos.to(device), labels.to(device), sums.to(device)
        
        optimizer.zero_grad()

        preds1,preds2 = model(images,F.one_hot(torch.tensor(randnos), num_classes=10).float()) # Pass Batch
        loss1 = F.cross_entropy(preds1, labels) # Calculate Loss
        loss2 = F.cross_entropy(preds2, sums) # Calculate Loss
        loss = (19*(loss1) + 10*(loss2))/29
        # loss = loss1 + loss2
        
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct_images += get_num_correct(preds1, labels)
        total_correct_sums += get_num_correct(preds2, sums)
    

    print( "epoch", epoch, 
        "total_correct(images):", total_correct_images,"/",len(train_ds),
        "total_correct(sum):" , total_correct_sums,"/",len(train_ds),
        "loss:", total_loss
    )


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch 0 total_correct(images): 22855 / 60000 total_correct(sum): 5665 / 60000 loss: 7807.050562977791
epoch 1 total_correct(images): 41702 / 60000 total_correct(sum): 5935 / 60000 loss: 5764.25578212738
epoch 2 total_correct(images): 47466 / 60000 total_correct(sum): 6548 / 60000 loss: 4982.4205040335655
epoch 3 total_correct(images): 49880 / 60000 total_correct(sum): 8180 / 60000 loss: 4508.7538805007935
epoch 4 total_correct(images): 51578 / 60000 total_correct(sum): 9660 / 60000 loss: 4060.484685897827
epoch 5 total_correct(images): 52944 / 60000 total_correct(sum): 11323 / 60000 loss: 3693.761387348175
epoch 6 total_correct(images): 54010 / 60000 total_correct(sum): 13394 / 60000 loss: 3389.1274438500404
epoch 7 total_correct(images): 54736 / 60000 total_correct(sum): 16223 / 60000 loss: 3133.316538631916
epoch 8 total_correct(images): 55339 / 60000 total_correct(sum): 18751 / 60000 loss: 2928.797610759735
epoch 9 total_correct(images): 55724 / 60000 total_correct(sum): 21089 / 600

In [14]:
# validate the model #
model.eval()
for test_batch in test_loader: # Get Batch
    images_test, randnos_test, labels_test, sums_test = test_batch
    images_test, randnos_test, labels_test, sums_test = images_test.to(device), randnos_test.to(device), labels_test.to(device), sums_test.to(device)
    test_preds1,test_preds2 = model(images_test,F.one_hot(torch.tensor(randnos_test), num_classes=10).float())
    loss1_t = F.cross_entropy(test_preds1, labels_test) # Calculate Loss
    loss2_t = F.cross_entropy(test_preds2, sums_test) # Calculate Loss
    loss_t = (19*(loss1_t) + 10*(loss2_t))/29
    total_loss_t += loss_t.item()
    total_correct_images_t += get_num_correct(test_preds1, labels_test)
    total_correct_sums_t += get_num_correct(test_preds2, sums_test)

print(  "total_correct(images):", total_correct_images_t,"/",len(test_ds),
        "total_correct(sum):" , total_correct_sums_t,"/",len(test_ds),
        "loss:", total_loss_t
    )


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


total_correct(images): 9763 / 10000 total_correct(sum): 9637 / 10000 loss: 72.99444600567222
